<img src="https://www.mines.edu/webcentral/wp-content/uploads/sites/267/2019/02/horizontallightbackground.jpg" width="100%"> 

### CSCI250 Python Computing: Building a Sensor System
<hr style="height:5px" width="100%" align="left">

# Microphone

# Introduction

<img src="https://cdn.sparkfun.com//assets/parts/1/0/5/7/0/14262-01.jpg" width="30%" align="right">

The sound detector consists of a **microphone** producing voltages that depend on the air pressure variation caused by sound waves. 

Such sensors use a magnet attached to a thin membrane. Pressure waves hitting the membrane cause the magnet to move in and out of a coil of wire, thus producing a voltage by induction. 

You can learn more about microphones on [this page](https://en.wikipedia.org/wiki/Microphone).

# Wiring

The sound detector connects through several pins:
* the audio pin
* the envelope pin
* the gate pin

*** 
See [SparkFun's hookup guide](https://learn.sparkfun.com/tutorials/sound-detector-hookup-guide) for more information.

## the audio pin 
**analog output**: returns raw voltage from the microphone

* Connect by the [ADC](h_ADC.ipynb) (cannot measure negative voltage).
* The audio **is biased** by the supply voltage 
    * it returns voltage around ~$1.65V$

## the envelope pin
**analog output**: returns a voltage that tracks the audio peaks

* Connect by the [ADC](h_ADC.ipynb) (cannot measure negative voltage).
* The envelope pin **is not biased** by the supply voltage
    * when it's quiet, the envelope outputs ~$0$V

<img src="https://upload.wikimedia.org/wikipedia/commons/4/4d/C_Envelope_follower.png" width="100%">

## the gate pin
**digital output**: returns 1 when sound is detected; 0 otherwise

* Connect by [GPIO](h_GPIO.ipynb).
* It detects when the envelope goes above some threshold.
* Can also be synthesized in software from the envelope.

The remaining pins are **VCC** and **GND**, which are connected to $3.3V$ and GND on the wedge, respectively.

**N.B.**: the wiring for the ADC is omitted for clarity.

<img src="https://www.dropbox.com/s/3cxkbj3lr40jm4g/wiring_sound_detector.PNG?raw=1" align="left" width="70%">

<img src="https://www.dropbox.com/s/8s6fax3mmr0c0d3/circuit_sound_detector.jpg?raw=1" align="right" width="30%">

# Programming

Follow the [ADC notebook](h_ADC.ipynb) to set-up the analog/digital converter. 

In [1]:
import adcUtil as adc

The voltages measured from the audio pin represent  samples of a continually changing time series. 

Confirm the behavior of the pins:
* the audio voltage is ~$1.65V$ when it is quiet
* the envelope returns higher voltage when it gets louder
* the gate goes HIGH when the envelope passes a threshold

Take many measurements and plot the data to see the waveform.

In [3]:
# import GPIO library
import RPi.GPIO as GPIO

# setup GPIO as input for the gate pin
GPIO.setmode(GPIO.BCM)
gatePin = 21
GPIO.setup(gatePin, GPIO.IN)

In [6]:
import time

to = time.time()
t = to
while t < to + 10: # loop for 10s
    gateState = GPIO.input(gatePin)              #     gate channel (digital)
    Vaudio    = adc.readADC(channel=0, device=0) #    audio channel (analog)
    Venvelope = adc.readADC(channel=1, device=0) # envelope channel (analog)
    
    print(format(     t-to,'.2f'),
          format(gateState,  'd'),
          format(   Vaudio,'.3f'),
          format(Venvelope,'.3f'))
    
    time.sleep(0.5)
    t = time.time()

0.00 1 1.655 0.045
0.51 1 1.629 0.042
1.01 0 1.652 0.039
1.51 0 1.648 0.039
2.01 1 1.642 0.042
2.51 1 1.648 0.039
3.02 1 1.471 0.877
3.52 1 1.613 0.406
4.02 1 1.603 0.245
4.52 0 1.681 0.084
5.02 0 1.655 0.177
5.53 0 1.645 0.045
6.03 1 1.645 0.039
6.53 1 1.642 0.039
7.03 1 1.635 0.042
7.53 1 1.655 0.039
8.03 0 1.652 0.039
8.54 0 1.648 0.039
9.04 0 1.648 0.039
9.54 0 1.635 0.042


<img src="http://www.dropbox.com/s/fcucolyuzdjl80k/todo.jpg?raw=1" width="10%" align="left">

Test the behavior of the microphone by running the test code and speaking close to the sensor.

In [7]:
# return GPIO to a base state
GPIO.cleanup()

# Troubleshooting

* **I get error messages when I run my code.**
    * There's probably an issue with your code, not any of the components. Read the error message and try to understand what it's telling you in order to fix it.

* **The ADC always returns 0.**
    * You're probably having an issue communicating with the ADC, which would most likely come from a wiring problem. Double check your wiring of the ADC to make sure the connections are correct. Also double check your code for any bugs, and that the channel you're measuring isn't just connected to 0V.
    * Make sure your sensor is wired correctly. The outer pins need to be connected to 3.3V and GND, and the center pin connected to one of the ADC channel pins.
    
* **I get funky numbers from the ADC when I read from the sound detector.**
    * Because the sound detector is essentially a microphone, the output is always going to be a bit noisy because there's always some audio that the microphone picks up (unless you're in an anechoic chamber). If you think it's something other than normal audio noise, try connecting something else to the ADC like a potentiometer. If you still get abnormal values with the potentiometer, then there is likely an issue with the ADC (also make sure the measurement pin isn't floating). You can double check by swapping your ADC with another.
    * If the ADC is working fine, then your sound detector is probably also fine. If you want to double check this, swap your sound detector with your neighbor's. If you get similar data, then there probably isn't an issue with your sound detector.

<img src="https://www.dropbox.com/s/wj23ce93pa9j8pe/demo.png?raw=1" width="10%" align="left">

# Exercise

Capture and display the raw data from all the microphone pins. Make sound close to the microphone to produce obvious results, and plot the data.